In [1]:
library(RNifti)
library(dplyr)
library(oro.nifti)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


oro.nifti 0.11.4


Attaching package: ‘oro.nifti’


The following object is masked from ‘package:dplyr’:

    slice


The following object is masked from ‘package:RNifti’:

    origin




In [2]:
results_dir <- "scratch/tractoflow_hcp_dwi/results/"
patients <- list.files(results_dir, full.names = TRUE)
patients <- patients[grep("^\\d{6}$", basename(patients))]
patient_data <- read.csv("scratch/tractoflow_hcp_dwi/patient_data.csv")
patient_data_gender <- read.csv("scratch/tractoflow_hcp_dwi/patient_data_gender.csv")
subj_data <- patient_data[c("Subject", "Age_in_Yrs", "BMI", "SSAGA_Income", "SSAGA_Educ", "BPSystolic", "BPDiastolic", "HbA1C")]
subj_data_gender <- patient_data_gender[c("Subject", "Gender")]
subj_data=merge(subj_data, subj_data_gender)
rois <- c("caudate_spm", "hippocampus_spm", "nacc_spm", "pallidum_spm", "putamen_spm")

In [3]:
roi_data <- data.frame(Subject=character(0))
for (i in 1:length(patients)) {
    var_names <-list()
    data <-list()
    avg_vals <- list()
    for (roi in 1:length(rois)) {
        if (i==1) {
            roi_data[[rois[roi]]] <- numeric(0)
        }
        var_names[[roi]] <- file.path("scratch", "tractoflow_hcp_dwi", "subject_other_rois", basename(patients[i]), "ad_rois", paste0(basename(patients[i]), "_ad_", rois[roi], "_bin_resampled.nii.gz"))
        data[[roi]] <- readNifti(var_names[[roi]])
        non_zero_vals <- data[[roi]]
        non_zero_vals <- non_zero_vals[non_zero_vals > 0]
        avg_val <- mean(non_zero_vals)
        avg_vals <- append(avg_vals, avg_val)
        temp_data <- data.frame(Subject=basename(patients[i]), avg_vals)
    }
    colnames(temp_data)=c("Subject", rois)
    roi_data <- rbind(roi_data, temp_data)
}
subj_data <- merge(subj_data, roi_data)
write.csv(subj_data, "scratch/tractoflow_hcp_dwi/linear_regression/ad_other_rois.csv")